In [203]:
import numpy as np
from collections import defaultdict
import threading
import queue
arr = np.genfromtxt('Day23.txt',delimiter = ',',dtype=np.int64)

In [204]:
def indexing(listt,index):
    inxlist = []
    global arr1
    for i, ss in enumerate(listt):
        if ss == '0': #position mode
            inxlist.append(arr1[index+i+1])
        elif ss =='1': #immediate mode
            inxlist.append(index+i+1)
        else: #ss == '2'
            global base
            inxlist.append(arr1[index+i+1]+base)
    return inxlist

def arrOdict(inxlist):
    global arr1
    global lgmem
    vallist = []
    for inx in inxlist:
        if inx > (len(arr1)-1):
            if inx not in lgmem.keys():
                lgmem[inx] = 0
                vallist.append(lgmem[inx])
            else:
                vallist.append(lgmem[inx])
        else:
            vallist.append(arr1[inx])
    return vallist

def setequal(inxequal,num):
    global arr1
    global lgmem
    if inxequal > (len(arr1)-1):
        lgmem[inxequal] = num
    else:
        arr1[inxequal] = num
    return None

def opcodeAct(index,inxlist,opcode):
    global arr1
    global output_tracker
    global idd
    global inpt
    vallist = arrOdict(inxlist)
    if opcode == 1:
        setequal(inxlist[2],vallist[0] + vallist[1])
        index += 4
    elif opcode == 2:
        setequal(inxlist[2],vallist[0] * vallist[1])
        index += 4
    elif opcode == 4:
        output_tracker[idd].append(vallist[0])
        index += 2
    elif opcode == 3:
        setequal(inxlist[0],inpt)
        inpt = -1
        index += 2
    elif opcode == 5:
        if vallist[0]: #If non-zero is true
            index = vallist[1]
        else:
            index += 3
    elif opcode == 6:
        if not vallist[0]:
            index = vallist[1]
        else:
            index += 3
    elif opcode == 7:
        if vallist[0] < vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 8:
        if vallist[0] == vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 9:
        global base
        base += vallist[0]
        index += 2
    return index

def day9(index):
    global arr1
    global output_tracker
    global idd
    if arr1[index] in [1,2,7,8]:
        index = opcodeAct(index,arr1[index+1:index+4],arr1[index])
    elif arr1[index] in [5,6]:
        index = opcodeAct(index,arr1[index+1:index+3],arr1[index])
    elif arr1[index] in [3,4,9]:
        index = opcodeAct(index,arr1[index+1:index+2],arr1[index])
    else: #5 digit opcode
        s = str(arr1[index])
        s = s.rjust(5,'0')
        s = s[::-1]
        if int(s[0]) in [1,2,7,8]:
            inxlist = indexing(s[2:],index)
        elif int(s[0]) in [5,6]:
            inxlist = indexing(s[2:4],index)
        elif int(s[0]) in [3,4,9]:
            inxlist = indexing(s[2:3],index)
        index = opcodeAct(index,inxlist,int(s[0]))
    return index

In [264]:
global output_tracker
global idd
output_tracker = defaultdict(list)
output_tracker.update({i:[] for i in range(50)})
inpt_tracker = defaultdict(list)
inpt_tracker.update({i:[] for i in range(50)})
inpt_tracker[255] = []

base_queue = queue.Queue()
lgmem_queue = queue.Queue()
arr_queue = queue.Queue()
index_queue = queue.Queue()
id_queue = queue.Queue()
inpt_queue = queue.Queue()

for idd in range(50):
    base = 0
    lgmem = {}
    arr1 = arr.copy()
    inpt = idd
    index = day9(0)
    base_queue.put(base)
    lgmem_queue.put(lgmem)
    arr_queue.put(arr1)
    index_queue.put(index)
    id_queue.put(idd)
    inpt_queue.put(inpt)
    
while len(inpt_tracker[255]) != 2:
    base = base_queue.get()
    lgmem = lgmem_queue.get()
    arr1 = arr_queue.get()
    index = index_queue.get()
    idd = id_queue.get()
    inpt = inpt_queue.get()
    if len(output_tracker[idd]) == 3:
        inpt_tracker[output_tracker[idd][0]] += output_tracker[idd][1:]
        output_tracker[idd] = []
    if (inpt_tracker[idd] != []) & (inpt == -1):
        inpt = inpt_tracker[idd].pop(0)
    index = day9(index)
    if arr1[index] != 99:
        base_queue.put(base)
        lgmem_queue.put(lgmem)
        arr_queue.put(arr1)
        index_queue.put(index)
        id_queue.put(idd)
        inpt_queue.put(inpt)
inpt_tracker[255][1]

16660

In [322]:
#Part 2
global output_tracker
global idd
output_tracker = defaultdict(list)
output_tracker.update({i:[] for i in range(50)})
inpt_tracker = defaultdict(list)
inpt_tracker.update({i:[] for i in range(50)})
nat = []

base_queue = queue.Queue()
lgmem_queue = queue.Queue()
arr_queue = queue.Queue()
index_queue = queue.Queue()
id_queue = queue.Queue()
inpt_queue = queue.Queue()

for idd in range(50):
    base = 0
    lgmem = {}
    arr1 = arr.copy()
    inpt = idd
    index = day9(0)
    base_queue.put(base)
    lgmem_queue.put(lgmem)
    arr_queue.put(arr1)
    index_queue.put(index)
    id_queue.put(idd)
    inpt_queue.put(inpt)
    
previous = 0
current = 1
counter = 0
while previous != current:
    base = base_queue.get()
    lgmem = lgmem_queue.get()
    arr1 = arr_queue.get()
    index = index_queue.get()
    idd = id_queue.get()
    inpt = inpt_queue.get()
    if len(output_tracker[idd]) == 3:
        if output_tracker[idd][0] == 255:
            nat = output_tracker[idd][1:]
        else:
            inpt_tracker[output_tracker[idd][0]] += output_tracker[idd][1:]
        output_tracker[idd] = []
    if (inpt_tracker[idd] != []) & (inpt == -1):
        inpt = inpt_tracker[idd].pop(0)
    index = day9(index)
    if arr1[index] != 99:
        base_queue.put(base)
        lgmem_queue.put(lgmem)
        arr_queue.put(arr1)
        index_queue.put(index)
        id_queue.put(idd)
        inpt_queue.put(inpt)
    if (not any(inpt_tracker.values())) & (not any(output_tracker.values())) & (set(inpt_queue.queue) == {-1}) & (counter > 65000):
        previous = current
        inpt_tracker[0] = nat
        current = nat[1]
        counter = 0

IndexError: list index out of range

In [323]:
nat[1]

1001